# Lab 6: Foundry IQ - Knowledge Retrieval for Agents

Build an agent that answers questions about **Space Facts** using Foundry IQ!

## What is Foundry IQ?

| Without Foundry IQ | With Foundry IQ |
|-------------------|----------------|
| Embed RAG logic in every agent | Centralized knowledge retrieval |
| Duplicate retrieval configs | Share knowledge bases across agents |
| Manual query decomposition | Automatic query planning & synthesis |


## Features Demonstrated
- **APIM Gateway** - Uses central Landing Zone models (chat + embeddings)
- **Vector Search** - Embeddings via APIM for semantic similarity
- **Multiple knowledge sources** in one knowledge base
- **Search index** with custom CSV data + vector embeddings

## Prerequisites
- Completed **Lab 1a** (Landing Zone with APIM + embedding model) 
- `.env` file with APIM_URL and APIM_KEY

## Step 1: Install Dependencies

In [1]:
!pip install pandas requests azure-ai-projects==2.0.0b2 azure-identity azure-search-documents -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


## Step 2: Configure Variables

In [ ]:
import subprocess
import os
from pathlib import Path
from IPython.display import display, Markdown

# Load .env from parent directory
env_path = Path("../.env")
if env_path.exists():
    for line in env_path.read_text().splitlines():
        if '=' in line and not line.startswith('#'):
            key, value = line.split('=', 1)
            os.environ[key.strip()] = value.strip()

# Landing Zone settings (from Lab 1a)
APIM_URL = os.environ.get("APIM_URL", "")
APIM_KEY = os.environ.get("APIM_KEY", "")
MODEL_NAME = os.environ.get("MODEL_NAME", "gpt-4.1-mini")
EMBEDDING_MODEL = os.environ.get("EMBEDDING_MODEL", "text-embedding-3-large")

# Resource group for this lab
RG = "foundryiq-lab"
LOCATION = "eastus2"

# Names for our Foundry IQ resources
KNOWLEDGE_BASE = "space-facts-kb"

# Get current user info
PRINCIPAL_ID = subprocess.run(
    'az ad signed-in-user show --query id -o tsv',
    shell=True, capture_output=True, text=True
).stdout.strip()

SUBSCRIPTION_ID = subprocess.run(
    'az account show --query id -o tsv',
    shell=True, capture_output=True, text=True
).stdout.strip()

# Verify Landing Zone is configured
if not APIM_URL or not APIM_KEY:
    print("❌ Missing APIM_URL or APIM_KEY in .env file!")
    print("   Please complete Lab 1a first to deploy the Landing Zone")
else:
    display(Markdown(f'''
### ✅ Configuration Loaded

| Setting | Value |
|---------|-------|
| APIM Gateway | `{APIM_URL[:50]}...` |
| Chat Model | `{MODEL_NAME}` |
| Embedding Model | `{EMBEDDING_MODEL}` |
| Resource Group | `{RG}` |
| Knowledge Base | `{KNOWLEDGE_BASE}` |
'''))

## Step 3: Create Resource Group & Deploy Spoke

This creates:
- **Azure AI Search** (for Foundry IQ knowledge bases)
- **AI Foundry Account + Project** (uses APIM gateway - no local models!)
- **APIM Connection** to Landing Zone
- All necessary RBAC permissions

⏱️ ~5 minutes

In [3]:
!az group create -n "{RG}" -l "{LOCATION}" -o table

Location    Name
----------  -------------
eastus2     foundryiq-lab


In [ ]:
!az deployment group create -g "{RG}" --template-file spoke.bicep \
    -p deployerPrincipalId="{PRINCIPAL_ID}" \
    -p apimUrl="{APIM_URL}" \
    -p apimSubscriptionKey="{APIM_KEY}" \
    -p gatewayModelName="{MODEL_NAME}" \
    -o table

In [ ]:
import json

# Get deployment outputs
outputs = json.loads(subprocess.run(
    f'az deployment group show -g "{RG}" -n spoke --query properties.outputs -o json',
    shell=True, capture_output=True, text=True
).stdout)

ACCOUNT_NAME = outputs['accountName']['value']
PROJECT_NAME = outputs['projectName']['value']
PROJECT_ENDPOINT = outputs['projectEndpoint']['value']
PROJECT_MI = outputs['projectManagedIdentityId']['value']
APIM_CONNECTION = outputs['apimConnectionName']['value']
SEARCH_ENDPOINT = outputs['searchEndpoint']['value']
SEARCH_NAME = outputs['searchName']['value']

# Gateway model must be in format: <connection-name>/<model-name>
GATEWAY_MODEL = f"{APIM_CONNECTION}/{outputs['gatewayModelName']['value']}"

display(Markdown(f'''
### ✅ Spoke Deployment Complete!

| Resource | Value |
|----------|-------|
| AI Account | `{ACCOUNT_NAME}` |
| Project | `{PROJECT_NAME}` |
| APIM Connection | `{APIM_CONNECTION}` |
| Gateway Model | `{GATEWAY_MODEL}` |
| Search Service | `{SEARCH_NAME}` |

💡 **No local model deployments** - using APIM gateway to Landing Zone!
'''))

## Step 4: Wait for RBAC Propagation

Azure role assignments can take a minute to propagate.

In [39]:
from IPython.display import clear_output
import time

for i in range(60, 0, -10):
    clear_output(wait=True)
    print(f"⏳ Waiting for RBAC to propagate... {i}s")
    time.sleep(10)

clear_output(wait=True)
print("✅ RBAC permissions ready!")

✅ RBAC permissions ready!


## Step 5: Create Search Index with Vector Search

We'll create a search index with **vector search** capabilities and load fun space facts from a CSV file.

The embedding model runs through the APIM gateway (same API key as chat completions)!

In [6]:
from azure.identity import DefaultAzureCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex, SearchField, SearchFieldDataType,
    SemanticConfiguration, SemanticField, SemanticPrioritizedFields, SemanticSearch,
    VectorSearch, HnswAlgorithmConfiguration, VectorSearchProfile
)

credential = DefaultAzureCredential()
index_client = SearchIndexClient(endpoint=SEARCH_ENDPOINT, credential=credential)

INDEX_NAME = "space-facts"

# Create index with vector search + semantic search
index = SearchIndex(
    name=INDEX_NAME,
    fields=[
        SearchField(name="id", type="Edm.String", key=True, filterable=True),
        SearchField(name="fact", type="Edm.String", searchable=True),
        SearchField(name="category", type="Edm.String", filterable=True, facetable=True),
        SearchField(name="fun_rating", type="Edm.Int32", filterable=True, sortable=True),
        # Vector field for embeddings (text-embedding-3-large = 3072 dimensions)
        SearchField(
            name="fact_vector", 
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            vector_search_dimensions=3072,
            vector_search_profile_name="vector-profile",
            searchable=True
        ),
    ],
    vector_search=VectorSearch(
        algorithms=[HnswAlgorithmConfiguration(name="hnsw-algo")],
        profiles=[VectorSearchProfile(name="vector-profile", algorithm_configuration_name="hnsw-algo")]
    ),
    semantic_search=SemanticSearch(
        default_configuration_name="semantic-config",
        configurations=[SemanticConfiguration(
            name="semantic-config",
            prioritized_fields=SemanticPrioritizedFields(
                content_fields=[SemanticField(field_name="fact")]
            )
        )]
    )
)

index_client.create_or_update_index(index)
print(f"✅ Index '{INDEX_NAME}' created with vector search + semantic search!")

✅ Index 'space-facts' created with vector search + semantic search!


In [7]:
import pandas as pd
import requests
from azure.search.documents import SearchClient

# Create embedding function using APIM gateway
def get_embedding(text: str) -> list:
    """Get embedding via APIM gateway."""
    response = requests.post(
        f"{APIM_URL}/deployments/{EMBEDDING_MODEL}/embeddings?api-version=2024-10-21",
        headers={"api-key": APIM_KEY, "Content-Type": "application/json"},
        json={"input": text, "model": EMBEDDING_MODEL}
    )
    response.raise_for_status()
    return response.json()["data"][0]["embedding"]

# Test the embedding endpoint
test_embedding = get_embedding("Hello space!")
print(f"✅ Embedding model working via APIM! Dimension: {len(test_embedding)}")

# Load space facts from CSV
df = pd.read_csv("space_facts.csv")
print(f"📚 Loaded {len(df)} space facts from CSV")
display(df.head())

✅ Embedding model working via APIM! Dimension: 3072
📚 Loaded 15 space facts from CSV


,id,fact,category,fun_rating
0,fact-001,A day on Venus is longer than its year! Venus ...,planets,5
1,fact-002,Jupiter's Great Red Spot is a storm that has b...,planets,5
2,fact-003,Saturn's rings are made mostly of ice particle...,planets,4
3,fact-004,Neutron stars are so dense that a teaspoon of ...,stars,5
4,fact-005,The Sun contains 99.86% of all mass in our sol...,stars,4


## Step 6: Upload Documents with Vector Embeddings

In [8]:
from azure.search.documents import SearchIndexingBufferedSender

# Convert CSV facts to documents with embeddings
print("🔄 Generating embeddings for space facts...")
documents = []
for i, row in df.iterrows():
    fact_text = row["fact"]
    documents.append({
        "id": row["id"],
        "fact": fact_text,
        "category": row["category"],
        "fun_rating": int(row["fun_rating"]),
        "fact_vector": get_embedding(fact_text),
    })
    print(f"  ✓ Fact {i+1}/{len(df)}", end="\r")

print(f"\n✅ Generated embeddings for {len(df)} facts")

# Upload all documents
with SearchIndexingBufferedSender(endpoint=SEARCH_ENDPOINT, index_name=INDEX_NAME, credential=credential) as sender:
    sender.upload_documents(documents=documents)

print(f"✅ Uploaded {len(documents)} facts with vector embeddings")

🔄 Generating embeddings for space facts...
  ✓ Fact 15/15
✅ Generated embeddings for 15 facts
✅ Uploaded 15 facts with vector embeddings


## Step 7: Create Knowledge Source

We'll create a knowledge source from our search index.

In [9]:
from iq_helpers import FoundryIQClient
from display_helpers import show_success, show_error

iq = FoundryIQClient(SEARCH_ENDPOINT)

# Create search index knowledge source
INDEX_SOURCE = "space-index-source"
result = iq.create_knowledge_source(
    name=INDEX_SOURCE,
    kind="searchIndex",
    config={
        "searchIndexParameters": {
            "searchIndexName": INDEX_NAME,
            "semanticConfigurationName": "semantic-config",
            "sourceDataFields": [],
            "searchFields": []
        }
    }
)
if 'error' not in result:
    show_success(f"Knowledge source '{INDEX_SOURCE}' created!")
else:
    show_error(result.get('error', 'Unknown error'))

### ✅ Knowledge source 'space-index-source' created!

## Step 8: Create Knowledge Base 

The knowledge base orchestrates retrieval from our knowledge source.

In [10]:
# Create knowledge base with index source + APIM model for reasoning
APIM_BASE_URL = APIM_URL.replace('/openai', '')

model_config = {
    "kind": "azureOpenAI",
    "azureOpenAIParameters": {
        "resourceUri": APIM_BASE_URL,
        "deploymentId": MODEL_NAME,
        "apiKey": APIM_KEY,
        "modelName": MODEL_NAME
    }
}

result = iq.create_knowledge_base(
    name=KNOWLEDGE_BASE,
    sources=[INDEX_SOURCE],
    description="Space facts from curated CSV data",
    model_config=model_config
)

if 'error' not in result:
    show_success(f"Knowledge base '{KNOWLEDGE_BASE}' created with APIM model for reasoning!")
else:
    show_error(result['error'])

### ✅ Knowledge base 'space-facts-kb' created with APIM model for reasoning!

## Step 9: Test Direct Queries

Let's query the knowledge base directly before connecting to an agent.

In [11]:
from display_helpers import show_query_result

# Query from CSV data
result = iq.query_knowledge_base(KNOWLEDGE_BASE, "What is the largest volcano in the solar system?")
show_query_result("What is the largest volcano in the solar system?", result)

**Query:** *"What is the largest volcano in the solar system?"*

---

**Answer:**

[{"ref_id":0,"content":"Mars has the largest volcano in the solar system called Olympus Mons which is about 13.6 miles high."},{"ref_id":1,"content":"A year on Mercury is just 88 Earth days but a day on Mercury lasts 59 Earth days."},{"ref_id":2,"content":"A day on Venus is longer than its year! Venus takes 243 Earth days to rotate once but only 225 Earth days to orbit the Sun."},{"ref_id":3,"content":"Jupiter's Great Red Spot is a storm that has been raging for over 400 years and is so big that Earth could fit inside it."},{"ref_id":4,"content":"Footprints on the Moon will last for millions of years because there is no wind or water to erode them."}]


**📖 References:** 5 source(s) used

In [12]:
# Another test query
result = iq.query_knowledge_base(KNOWLEDGE_BASE, "Tell me about Jupiter's storm")
show_query_result("Tell me about Jupiter's storm", result)

**Query:** *"Tell me about Jupiter's storm"*

---

**Answer:**

[{"ref_id":0,"content":"Jupiter's Great Red Spot is a storm that has been raging for over 400 years and is so big that Earth could fit inside it."},{"ref_id":1,"content":"The International Space Station travels at about 17500 mph completing one orbit around Earth every 90 minutes."},{"ref_id":2,"content":"Neutron stars are so dense that a teaspoon of their material would weigh about 6 billion tons on Earth."}]


**📖 References:** 3 source(s) used

## Step 10: Create MCP Connection 🔗

Connect the Foundry project to the knowledge base via MCP (Model Context Protocol).

In [13]:
from iq_helpers import create_mcp_connection

CONNECTION_NAME = "space-facts-mcp"

result = create_mcp_connection(
    subscription_id=SUBSCRIPTION_ID,
    resource_group=RG,
    account_name=ACCOUNT_NAME,
    project_name=PROJECT_NAME,
    connection_name=CONNECTION_NAME,
    search_endpoint=SEARCH_ENDPOINT,
    kb_name=KNOWLEDGE_BASE
)

if 'error' not in result:
    show_success(f"MCP connection '{CONNECTION_NAME}' created!")
else:
    show_error(f"{result.get('error', 'Unknown error')}")

### ✅ MCP connection 'space-facts-mcp' created!

## Step 11: Create Space Expert Agent 🤖

Now the fun part - create an agent that uses our knowledge base!

In [14]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, MCPTool

project_client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=credential)

# Build MCP endpoint URL for the knowledge base
mcp_endpoint = f"{SEARCH_ENDPOINT}/knowledgebases/{KNOWLEDGE_BASE}/mcp?api-version=2025-11-01-preview"

# Create the MCP tool
mcp_tool = MCPTool(
    server_label="space-facts",
    server_url=mcp_endpoint,
    require_approval="never",
    allowed_tools=["knowledge_base_retrieve"],
    project_connection_id=CONNECTION_NAME
)

# Create the agent (uses APIM gateway model!)
AGENT_NAME = "SpaceExpert"

agent = project_client.agents.create_version(
    agent_name=AGENT_NAME,
    definition=PromptAgentDefinition(
        model=GATEWAY_MODEL,
        instructions="""You are a knowledge retrieval agent.
Given a query, use the MCP tool to find relevant information from the space facts knowledge base.

Do not answer any questions without first consulting the MCP tool.

Give detailed citations in your answers.

If you can't find the answer in the mcp tool, respond with "I don't know. " (even if you know)

Refuse to answer any queries that are not related to space.
""",
        tools=[mcp_tool]
    )
)

print(f"✅ Agent '{agent.name}' v{agent.version} created!")
print(f"   Using model: {GATEWAY_MODEL} (via APIM gateway)")

✅ Agent 'SpaceExpert' v1 created!
   Using model: landing-zone-apim/gpt-4.1-mini (via APIM gateway)


## Step 12: Talk to Your Space Expert! 🚀

In [19]:
from display_helpers import show_agent_response

openai_client = project_client.get_openai_client()

def ask_space_expert(question: str):
    """Ask the space expert a question."""
    response = openai_client.responses.create(
        input=question + ", please provide citations.",
        extra_body={
            "agent": {
                "name": agent.name, 
                "version": agent.version, 
                "type": "agent_reference"
            }
        }
    )
    return response.output_text

In [20]:
# Ask about something from our CSV
answer = ask_space_expert("What's the largest storm in the solar system?")
show_agent_response("What's the largest storm in the solar system?", answer)


---
**🙋 You:** What's the largest storm in the solar system?

**🤖 Agent:** The largest storm in the solar system is Jupiter's Great Red Spot. It is a massive storm that has been raging for over 400 years and is so large that Earth could fit inside it. 

Citation: 
- "Jupiter’s Great Red Spot is a storm that has been raging for over 400 years and is so big that Earth could fit inside it." (fact-002)


In [21]:
# Ask about volcanoes
answer = ask_space_expert("What's the largest volcano in the solar system?")
show_agent_response("What's the largest volcano in the solar system?", answer)


---
**🙋 You:** What's the largest volcano in the solar system?

**🤖 Agent:** The largest volcano in the solar system is Olympus Mons on Mars. It stands about 13.6 miles (approximately 22 kilometers) high, making it the tallest known volcano in our solar system. 

Citation: "Mars has the largest volcano in the solar system called Olympus Mons which is about 13.6 miles high."【4:0†source】


In [22]:
# Ask something not in the knowledge base
answer = ask_space_expert("What's the capital of France?")
show_agent_response("What's the capital of France?", answer)


---
**🙋 You:** What's the capital of France?

**🤖 Agent:** I am specialized in space-related knowledge and cannot provide information about general topics like the capital of France. If you have any questions related to space, feel free to ask!


---
## 🎉 Summary

You built an AI agent with **vector search** and **APIM gateway integration** using Foundry IQ!

### Architecture

| Layer | Component | Purpose |
|-------|-----------|--------|
| **Landing Zone** | APIM Gateway | Central model access (chat + embeddings) |
| **Landing Zone** | text-embedding-3-large | Vector embeddings via APIM |
| **IQ Spoke** | AI Foundry Project | Agent hosting + APIM connection |
| **IQ Spoke** | Azure AI Search | Vector index + knowledge base hosting |
| **IQ Spoke** | Knowledge Base | Orchestrates retrieval from sources |
| **IQ Spoke** | Knowledge Source | Index with CSV data |

### Key Concepts

- **No local model deployments** = Cost savings, central governance
- **APIM Gateway** = Single point for model access, routing, policies
- **Vector Search** = Embeddings via APIM for semantic similarity
- **Foundry IQ** = Azure AI Search knowledge bases
- **MCP** = Model Context Protocol for tool connections

## Cleanup (Optional)

In [ ]:
# Uncomment to delete resources
# iq.delete_knowledge_base(KNOWLEDGE_BASE)
# iq.delete_knowledge_source(INDEX_SOURCE)
# index_client.delete_index(INDEX_NAME)
# !az group delete -n "{RG}" --yes --no-wait
# print("🗑️ Cleanup initiated (Search service deletion takes a few minutes)")